In [ ]:
import mindpype as mp

import numpy as np
from scipy import signal

In [ ]:
# Create a manual computation function to compare the output of the graph with
def manual_computation(input_data):
    
    # first filter the data
    sos = signal.butter(4,(8,35),btype='bandpass',output='sos',fs=250)
    filtered_data = signal.sosfilt(sos,input_data,axis=1)
    cov_mat = np.cov(filtered_data)
    
    return cov_mat

In [ ]:
# create a session
s = mp.Session.create()
trial_graph = mp.Graph.create(s)

In [ ]:
# Create dummy input data and tensor
input_data = np.random.randn(12,500)
input_tensor = mp.Tensor.create_from_data(s,(12,500),input_data)

# Create output tensor
output_tensor = mp.Tensor.create(s,(12,12))

# Create virtual tensor for intermediate output
intermediate_tensor = mp.Tensor.create_virtual(s)

In [ ]:
# create a filter
order = 4
bandpass = (8,35) # in Hz
fs = 250
filter_obj = mp.Filter.create_butter(s,order,bandpass,btype='bandpass',fs=fs,implementation='sos')

In [ ]:
# add the nodes
mp.kernels.FilterKernel.add_filter_node(trial_graph,input_tensor,filter_obj,intermediate_tensor)
mp.kernels.CovarianceKernel.add_covariance_node(trial_graph,intermediate_tensor,output_tensor)

In [ ]:
# verify the session (i.e. schedule the nodes)
sts = trial_graph.verify()

In [ ]:
# initializing the graph - not required since there are no nodes that require initialization/training data
sts = trial_graph.initialize()

In [ ]:
# RUN!
sts = trial_graph.execute()

In [ ]:
# compare the output with manual calculation
ground_truth = manual_computation(input_data)

max_diff = np.max(np.abs(output_tensor.data - ground_truth))
print(max_diff)

# Check if the difference is within the machine epsilon
if max_diff <= np.finfo(np.float64).eps:
    print("Test Passed =D")
else:
    print("Test Failed D=")
